In [1]:
import pandas as pd
import plotly.express as px
from Moeda import Moeda
from Acesso import Login
from Acentos import Acentuacao
from Query import Query

pd.set_option('display.max_columns',None)
pd.set_option('float_format','{:.2f}'.format)

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)


In [2]:
querys={
    
    'Estatico':
    
    """
    
    SELECT * FROM netfeira.vw_targetestatico
    WHERE [Tipo de Operação]='VENDAS'
    
    """,
    
    'Produtos':
    
    """
    
    SELECT * FROM netfeira.vw_produto
    
    """,
    
    'Cliente':
    
    """
    
    SELECT * FROM netfeira.vw_cliente
    
    """,
    
    'Segmento':
    
    """
    
    SELECT * FROM netfeira.vw_segmento
    
    """,
    
    'Calendario':
    
    """
    
    SELECT * FROM netfeira.vw_calend
    WHERE YEAR(Data)<>YEAR(GETDATE())
    
    """
    
    
}

In [3]:
df=sql.CriarTabela(kwargs=querys)

In [4]:
df['Estatico']=df['Estatico'].merge(df['Produtos'],on='SKU',how='inner')

In [5]:
df['Estatico']=df['Estatico'].loc[df['Estatico']['Fabricante'].str.contains('MCCAIN')]

In [6]:
df['Estatico']=df['Estatico'].merge(df['Calendario'],left_on='Data de Faturamento',right_on='Data',how='inner')

In [7]:
tabela='Anual'

anos=df['Calendario']['Ano'].unique().tolist()

colunas=['ID Cliente','Pedido']

df[tabela]=df['Estatico'].groupby(['Ano'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

for c in colunas:
    
    temp_df=pd.DataFrame()
    
    for a in anos:

        df['Temp']=df['Estatico'].loc[df['Estatico']['Ano']==a].groupby(['Ano',c,'Situação'],as_index=False).agg({'Total Venda':'sum'})

        df['Temp']=df['Temp'].groupby(['Ano','Situação'],as_index=False).agg({'Total Venda':'sum',c:'count'})

        df['Temp'].loc[df['Temp']['Total Venda']<0,c]*=-1

        df['Temp']=df['Temp'].groupby(['Ano'],as_index=False).agg({c:'sum'})
        
        temp_df=pd.concat([temp_df,df['Temp']],axis=0,ignore_index=True)
                
        pass
    
    df[tabela]=df[tabela].merge(temp_df,on='Ano',how='inner')
    
    pass

colunas=df[tabela].columns[1:]

# comparativo ano x ano
for c in colunas:
    
    for a in anos:
        
        try:
        
            total_atual=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

            total_anterior=df[tabela].loc[df[tabela]['Ano']==a-1,c].sum()

            perc=round((total_atual/total_anterior)-1,4)*100 if total_anterior>0 else 0

            df[tabela].loc[df[tabela]['Ano']==a,f'Perc {c}']=perc
            
            pass
        
        except:
            
            continue
        
        pass
    
    pass

# Ano autal x anos anteriores
for c in colunas:
    
    for a in anos:
        
        total_atual=df[tabela].loc[df[tabela]['Ano']==anos[-1],c].sum()
        
        total_anterior=df[tabela].loc[df[tabela]['Ano']==a,c].sum()
        
        perc=round((total_atual/total_anterior)-1,4)*100 if total_anterior>0 else 0
        
        df[tabela].loc[df[tabela]['Ano']==a,f'Anual {c}']=perc
        
        pass
    
    pass

df[tabela]

,Ano,Total Venda,Peso Bruto KG,ID Cliente,Pedido,Perc Total Venda,Perc Peso Bruto KG,Perc ID Cliente,Perc Pedido,Anual Total Venda,Anual Peso Bruto KG,Anual ID Cliente,Anual Pedido
0,2018,9505810.67,1229894.65,1442,29827,0.00,0.00,0.00,0.00,51.02,-22.22,31.48,-21.19
1,2019,11492372.60,1282567.75,1486,35119,20.90,4.28,3.05,17.74,24.91,-25.42,27.59,-33.07
2,2020,9884040.76,827540.03,1870,23570,-13.99,-35.48,25.84,-32.89,45.24,15.59,1.39,-0.27
3,2021,11050992.95,869726.35,1853,23947,11.81,5.10,-0.91,1.60,29.90,9.98,2.32,-1.84
4,2022,14355579.60,956552.49,1896,23506,29.90,9.98,2.32,-1.84,0.00,0.00,0.00,0.00


In [8]:
tabela='Anual'

ano_atual=df[tabela]['Ano'].max()

lista=[]

colunas=[l for l in df[tabela].columns.tolist() if str(l).find('Anual')>=0]

for c in colunas:
    
    val=df[tabela].loc[df[tabela][c]>0,'Ano'].unique().tolist()
    
    if len(val)<=0:
        
        continue
    
    print(f'Coluna: {c}\nLista: {val}')
    
    pass


Coluna: Anual Total Venda
Lista: [2018, 2019, 2020, 2021]
Coluna: Anual Peso Bruto KG
Lista: [2020, 2021]
Coluna: Anual ID Cliente
Lista: [2018, 2019, 2020, 2021]


In [9]:
tabela='Grupos'

df[tabela]=df['Estatico'].loc[df['Estatico']['Ano']==ano_atual].groupby(['Grupo MCCAIN'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

colunas=df[tabela].columns[1:]

for c in colunas:
    
    total=df[tabela][c].sum()
        
    for i in range(0,len(df[tabela])):
        
        total_geral=df[tabela].loc[i,c].sum()
        
        perc=round(total_geral/total,4)*100
        
        df[tabela].loc[i,f'Perc {c}']=perc
        
        pass
            
    pass

df[tabela]


,Grupo MCCAIN,Total Venda,Peso Bruto KG,Perc Total Venda,Perc Peso Bruto KG
0,MCCAIN FOOD,7255113.55,507815.38,50.54,53.09
1,MCCAIN VAREJO,7100466.05,448737.11,49.46,46.91


In [10]:
grupos=df['Grupos']['Grupo MCCAIN'].tolist()

tabela='Anual Grupos'

dados_df=pd.DataFrame()

for g in grupos:

    anos=df['Calendario']['Ano'].unique().tolist()

    colunas=['ID Cliente','Pedido']

    df[tabela]=df['Estatico'].loc[df['Estatico']['Grupo MCCAIN']==g].groupby(['Ano'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

    for c in colunas:

        temp_df=pd.DataFrame()

        for a in anos:

            df['Temp']=df['Estatico'].loc[(df['Estatico']['Ano']==a)&(df['Estatico']['Grupo MCCAIN']==g)].groupby(['Ano',c,'Situação'],as_index=False).agg({'Total Venda':'sum'})

            df['Temp']=df['Temp'].groupby(['Ano','Situação'],as_index=False).agg({'Total Venda':'sum',c:'count'})

            df['Temp'].loc[df['Temp']['Total Venda']<0,c]*=-1

            df['Temp']=df['Temp'].groupby(['Ano'],as_index=False).agg({c:'sum'})

            temp_df=pd.concat([temp_df,df['Temp']],axis=0,ignore_index=True)

            pass

        df[tabela]=df[tabela].merge(temp_df,on='Ano',how='inner')

        pass

    colunas=df[tabela].columns[1:]

    # comparativo ano x ano
    for c in colunas:

        for a in anos:

            try:

                total_atual=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

                total_anterior=df[tabela].loc[df[tabela]['Ano']==a-1,c].sum()

                perc=round((total_atual/total_anterior)-1,4)*100 if total_anterior>0 else 0

                df[tabela].loc[df[tabela]['Ano']==a,f'Perc {c}']=perc

                pass

            except:

                continue

            pass

        pass

    # Ano autal x anos anteriores
    for c in colunas:

        for a in anos:

            total_atual=df[tabela].loc[df[tabela]['Ano']==anos[-1],c].sum()

            total_anterior=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

            perc=round((total_atual/total_anterior)-1,4)*100 if total_anterior>0 else 0

            df[tabela].loc[df[tabela]['Ano']==a,f'Anual {c}']=perc

            pass

        pass
    
    print(g)
            
    df[tabela]['Grupo MCCAIN']=g
    
    dados_df=pd.concat([dados_df,df[tabela]],axis=0,ignore_index=True)

    display(df[tabela])
    
    pass

df[tabela]=dados_df
    

MCCAIN FOOD


,Ano,Total Venda,Peso Bruto KG,ID Cliente,Pedido,Perc Total Venda,Perc Peso Bruto KG,Perc ID Cliente,Perc Pedido,Anual Total Venda,Anual Peso Bruto KG,Anual ID Cliente,Anual Pedido,Grupo MCCAIN
0,2018,9505810.67,1229894.65,1442,29827,0.00,0.00,0.00,0.00,-23.68,-58.71,-8.25,-52.52,MCCAIN FOOD
1,2019,11492372.60,1282567.75,1486,35119,20.90,4.28,3.05,17.74,-36.87,-60.41,-10.97,-59.67,MCCAIN FOOD
2,2020,5865274.69,546604.90,1299,16497,-48.96,-57.38,-12.58,-53.03,23.70,-7.10,1.85,-14.15,MCCAIN FOOD
3,2021,5361724.31,433277.75,1071,13767,-8.59,-20.73,-17.55,-16.55,35.31,17.20,23.53,2.88,MCCAIN FOOD
4,2022,7255113.55,507815.38,1323,14163,35.31,17.20,23.53,2.88,0.00,0.00,0.00,0.00,MCCAIN FOOD


MCCAIN VAREJO


,Ano,Total Venda,Peso Bruto KG,ID Cliente,Pedido,Perc Total Venda,Perc Peso Bruto KG,Perc ID Cliente,Perc Pedido,Anual Total Venda,Anual Peso Bruto KG,Anual ID Cliente,Anual Pedido,Grupo MCCAIN
0,2020,4018766.07,280935.13,923,7563,0.00,0.00,0.00,0.00,76.68,59.73,21.13,32.10,MCCAIN VAREJO
1,2021,5689268.64,436448.61,1324,10825,41.57,55.36,43.45,43.13,24.80,2.82,-15.56,-7.70,MCCAIN VAREJO
2,2022,7100466.05,448737.11,1118,9991,24.80,2.82,-15.56,-7.70,0.00,0.00,0.00,0.00,MCCAIN VAREJO


In [11]:
tabela='Anual Grupos'

df[tabela].loc[(df[tabela]['Ano']>=ano_atual-2)&(df[tabela]['Anual Peso Bruto KG']<0)]

,Ano,Total Venda,Peso Bruto KG,ID Cliente,Pedido,Perc Total Venda,Perc Peso Bruto KG,Perc ID Cliente,Perc Pedido,Anual Total Venda,Anual Peso Bruto KG,Anual ID Cliente,Anual Pedido,Grupo MCCAIN
2,2020,5865274.69,546604.90,1299,16497,-48.96,-57.38,-12.58,-53.03,23.70,-7.10,1.85,-14.15,MCCAIN FOOD


In [12]:
col='Peso Bruto KG'

tabela='Temp'

df[tabela]=df['Estatico'].loc[(df['Estatico']['Grupo MCCAIN']=='MCCAIN FOOD')&(df['Estatico']['Ano'].isin([2020,2022]))].groupby(['Grupo MCCAIN','Produto','Linha','Ano'],as_index=False).agg({col:'sum'})

df[tabela]=df[tabela].pivot(index=['Grupo MCCAIN','Produto','Linha'],columns='Ano',values=col).reset_index()

df[tabela]=df[tabela].loc[~df[tabela][2020].isnull()]

colunas=[l for l in df[tabela].columns.tolist() if str(l).isnumeric()]

for c in colunas:
    
    df[tabela].loc[df[tabela][c].isnull(),c]=0
    
    pass

df[tabela]['Dif']=df[tabela][2022]-df[tabela][2020]

df[tabela]=df[tabela].loc[df[tabela]['Dif']<0]

df[tabela].sort_values('Dif',ascending=True,ignore_index=True,inplace=True)

df[tabela]=df[tabela].groupby(['Linha'],as_index=False).agg({'Dif':'sum'})

df[tabela].sort_values('Dif',ascending=True,ignore_index=True,inplace=True)

df[tabela]['Dif']=abs(df[tabela]['Dif'])

df[tabela].head()

Ano,Linha,Dif
0,BATATA GRADO B,90960.00
1,PALITO EXTRA CROCANT,39095.00
2,BATATA RUSTICA,15790.00
3,BATATA 7MM,13745.25
4,BTT PALITO COM CASCA,5970.00


In [13]:
df['Cliente']=df['Cliente'].merge(df['Segmento'],on='ID Segmento',how='inner')

In [14]:
df['Estatico']=df['Estatico'].merge(df['Cliente'],on='ID Cliente',how='inner')

In [15]:
tabela='Canais'

df[tabela]=df['Estatico'].loc[df['Estatico']['Ano']==ano_atual].groupby(['Canal'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

colunas=df[tabela].columns[1:]

for c in colunas:
    
    total=df[tabela][c].sum()
        
    for i in range(0,len(df[tabela])):
        
        total_geral=df[tabela].loc[i,c].sum()
        
        perc=round(total_geral/total,4)*100
        
        df[tabela].loc[i,f'Perc {c}']=perc
        
        pass
            
    pass

df[tabela]

,Canal,Total Venda,Peso Bruto KG,Perc Total Venda,Perc Peso Bruto KG
0,AS,7375939.81,470079.60,51.38,49.14
1,B2C,95801.60,10335.03,0.67,1.08
2,FS,6883838.19,476137.86,47.95,49.78


In [16]:
tabela='Canais Grupos'

df[tabela]=df['Estatico'].loc[df['Estatico']['Ano']==ano_atual].groupby(['Canal','Grupo MCCAIN'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

colunas=df[tabela].columns[2:]

for c in colunas:
    
    total=df[tabela][c].sum()
        
    for i in range(0,len(df[tabela])):
        
        total_geral=df[tabela].loc[i,c].sum()
        
        perc=round(total_geral/total,4)*100
        
        df[tabela].loc[i,f'Perc {c}']=perc
        
        pass
            
    pass

df[tabela]

,Canal,Grupo MCCAIN,Total Venda,Peso Bruto KG,Perc Total Venda,Perc Peso Bruto KG
0,AS,MCCAIN FOOD,643973.38,43881.46,4.49,4.59
1,AS,MCCAIN VAREJO,6731966.43,426198.14,46.89,44.56
2,B2C,MCCAIN FOOD,83408.44,7824.53,0.58,0.82
3,B2C,MCCAIN VAREJO,12393.16,2510.50,0.09,0.26
4,FS,MCCAIN FOOD,6527731.73,456109.39,45.47,47.68
5,FS,MCCAIN VAREJO,356106.46,20028.47,2.48,2.09


In [17]:
grupos=df['Canais']['Canal'].tolist()

tabela='Anual Canal'

dados_df=pd.DataFrame()

for g in grupos:

    anos=df['Calendario']['Ano'].unique().tolist()

    colunas=['ID Cliente','Pedido']

    df[tabela]=df['Estatico'].loc[df['Estatico']['Canal']==g].groupby(['Ano'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

    for c in colunas:

        temp_df=pd.DataFrame()

        for a in anos:

            df['Temp']=df['Estatico'].loc[(df['Estatico']['Ano']==a)&(df['Estatico']['Canal']==g)].groupby(['Ano',c,'Situação'],as_index=False).agg({'Total Venda':'sum'})

            df['Temp']=df['Temp'].groupby(['Ano','Situação'],as_index=False).agg({'Total Venda':'sum',c:'count'})

            df['Temp'].loc[df['Temp']['Total Venda']<0,c]*=-1

            df['Temp']=df['Temp'].groupby(['Ano'],as_index=False).agg({c:'sum'})

            temp_df=pd.concat([temp_df,df['Temp']],axis=0,ignore_index=True)

            pass

        df[tabela]=df[tabela].merge(temp_df,on='Ano',how='inner')

        pass

    colunas=df[tabela].columns[1:]

    # comparativo ano x ano
    for c in colunas:

        for a in anos:

            try:

                total_atual=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

                total_anterior=df[tabela].loc[df[tabela]['Ano']==a-1,c].sum()

                perc=round((total_atual/total_anterior)-1,4)*100 if total_anterior>0 else 0

                df[tabela].loc[df[tabela]['Ano']==a,f'Perc {c}']=perc

                pass

            except:

                continue

            pass

        pass

    # Ano autal x anos anteriores
    for c in colunas:

        for a in anos:

            total_atual=df[tabela].loc[df[tabela]['Ano']==anos[-1],c].sum()

            total_anterior=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

            perc=round((total_atual/total_anterior)-1,4)*100 if total_anterior>0 else 0

            df[tabela].loc[df[tabela]['Ano']==a,f'Anual {c}']=perc

            pass

        pass
    
    print(g)
            
    df[tabela]['Canal']=g
    
    dados_df=pd.concat([dados_df,df[tabela]],axis=0,ignore_index=True)

    display(df[tabela])
    
    pass

df[tabela]=dados_df

AS


,Ano,Total Venda,Peso Bruto KG,ID Cliente,Pedido,Perc Total Venda,Perc Peso Bruto KG,Perc ID Cliente,Perc Pedido,Anual Total Venda,Anual Peso Bruto KG,Anual ID Cliente,Anual Pedido,Canal
0,2018,510788.01,66181.23,264,2408,0.00,0.00,0.00,0.00,1344.03,610.29,212.12,314.00,AS
1,2019,700435.19,74218.12,255,2769,37.13,12.14,-3.41,14.99,953.05,533.38,223.14,260.02,AS
2,2020,4396387.01,315537.98,725,8130,527.67,325.15,184.31,193.61,67.77,48.98,13.66,22.62,AS
3,2021,5493118.47,403340.05,747,9271,24.95,27.83,3.03,14.03,34.28,16.55,10.31,7.53,AS
4,2022,7375939.81,470079.60,824,9969,34.28,16.55,10.31,7.53,0.00,0.00,0.00,0.00,AS


B2C


,Ano,Total Venda,Peso Bruto KG,ID Cliente,Pedido,Perc Total Venda,Perc Peso Bruto KG,Perc ID Cliente,Perc Pedido,Anual Total Venda,Anual Peso Bruto KG,Anual ID Cliente,Anual Pedido,Canal
0,2018,26520.05,3215.85,5,252,0.00,0.00,0.00,0.00,261.24,221.38,2400.00,90.87,B2C
1,2019,23052.53,2580.00,4,209,-13.08,-19.77,-20.00,-17.06,315.58,300.58,3025.00,130.14,B2C
2,2020,54527.43,4990.39,148,572,136.54,93.43,3600.00,173.68,75.69,107.10,-15.54,-15.91,B2C
3,2021,99720.70,19538.89,237,883,82.88,291.53,60.14,54.37,-3.93,-47.11,-47.26,-45.53,B2C
4,2022,95801.60,10335.03,125,481,-3.93,-47.11,-47.26,-45.53,0.00,0.00,0.00,0.00,B2C


FS


,Ano,Total Venda,Peso Bruto KG,ID Cliente,Pedido,Perc Total Venda,Perc Peso Bruto KG,Perc ID Cliente,Perc Pedido,Anual Total Venda,Anual Peso Bruto KG,Anual ID Cliente,Anual Pedido,Canal
0,2018,8968502.61,1160497.57,1173,27167,0.00,0.00,0.00,0.00,-23.24,-58.97,-19.27,-51.94,FS
1,2019,10768884.88,1205769.63,1227,32141,20.07,3.90,4.60,18.31,-36.08,-60.51,-22.82,-59.38,FS
2,2020,5433126.32,507011.65,997,14868,-49.55,-57.95,-18.74,-53.74,26.70,-6.09,-5.02,-12.19,FS
3,2021,5458153.78,446847.41,869,13793,0.46,-11.87,-12.84,-7.23,26.12,6.55,8.98,-5.34,FS
4,2022,6883838.19,476137.86,947,13056,26.12,6.55,8.98,-5.34,0.00,0.00,0.00,0.00,FS


In [18]:
tabela='Consolidado Anual'

dados_df=pd.DataFrame()

for canal in df['Canais']['Canal'].tolist():
        
    for g in df['Canais Grupos'].loc[df['Canais Grupos']['Canal']==canal,'Grupo MCCAIN'].unique().tolist():

        anos=df['Calendario']['Ano'].unique().tolist()

        colunas=['ID Cliente','Pedido']

        df[tabela]=df['Estatico'].loc[(df['Estatico']['Canal']==canal)&(df['Estatico']['Grupo MCCAIN']==g)].groupby(['Ano'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

        for c in colunas:

            temp_df=pd.DataFrame()

            for a in anos:

                df['Temp']=df['Estatico'].loc[(df['Estatico']['Ano']==a)&(df['Estatico']['Canal']==canal)&(df['Estatico']['Grupo MCCAIN']==g)].groupby(['Ano',c,'Situação'],as_index=False).agg({'Total Venda':'sum'})

                df['Temp']=df['Temp'].groupby(['Ano','Situação'],as_index=False).agg({'Total Venda':'sum',c:'count'})

                df['Temp'].loc[df['Temp']['Total Venda']<0,c]*=-1

                df['Temp']=df['Temp'].groupby(['Ano'],as_index=False).agg({c:'sum'})

                temp_df=pd.concat([temp_df,df['Temp']],axis=0,ignore_index=True)

                pass

            df[tabela]=df[tabela].merge(temp_df,on='Ano',how='inner')

            pass

        colunas=df[tabela].columns[1:]

        # comparativo ano x ano
        for c in colunas:

            for a in anos:

                try:

                    total_atual=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

                    total_anterior=df[tabela].loc[df[tabela]['Ano']==a-1,c].sum()

                    perc=round((total_atual/total_anterior)-1,4)*100 if total_anterior>0 else 0

                    df[tabela].loc[df[tabela]['Ano']==a,f'Perc {c}']=perc

                    pass

                except:

                    continue

                pass

            pass

        # Ano autal x anos anteriores
        for c in colunas:

            for a in anos:

                total_atual=df[tabela].loc[df[tabela]['Ano']==anos[-1],c].sum()

                total_anterior=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

                perc=round((total_atual/total_anterior)-1,4)*100 if total_anterior>0 else 0

                df[tabela].loc[df[tabela]['Ano']==a,f'Anual {c}']=perc

                pass

            pass
        
        print(f'Canal: {canal} x {g}')
        
        df[tabela]['Grupo MCCAIN']=g
        
        df[tabela]['Canal']=canal
        
        dados_df=pd.concat([dados_df,df[tabela]],axis=0,ignore_index=True)
        
        display(df[tabela])
        
        pass
        
    pass

df[tabela]=dados_df

Canal: AS x MCCAIN FOOD


,Ano,Total Venda,Peso Bruto KG,ID Cliente,Pedido,Perc Total Venda,Perc Peso Bruto KG,Perc ID Cliente,Perc Pedido,Anual Total Venda,Anual Peso Bruto KG,Anual ID Cliente,Anual Pedido,Grupo MCCAIN,Canal
0,2018,510788.01,66181.23,264,2408,0.00,0.00,0.00,0.00,26.07,-33.70,24.62,-37.87,MCCAIN FOOD,AS
1,2019,700435.19,74218.12,255,2769,37.13,12.14,-3.41,14.99,-8.06,-40.88,29.02,-45.97,MCCAIN FOOD,AS
2,2020,520630.18,49356.09,249,1739,-25.67,-33.50,-2.35,-37.20,23.69,-11.09,32.13,-13.97,MCCAIN FOOD,AS
3,2021,410358.80,37046.60,181,1345,-21.18,-24.94,-27.31,-22.66,56.93,18.45,81.77,11.23,MCCAIN FOOD,AS
4,2022,643973.38,43881.46,329,1496,56.93,18.45,81.77,11.23,0.00,0.00,0.00,0.00,MCCAIN FOOD,AS


Canal: AS x MCCAIN VAREJO


,Ano,Total Venda,Peso Bruto KG,ID Cliente,Pedido,Perc Total Venda,Perc Peso Bruto KG,Perc ID Cliente,Perc Pedido,Anual Total Venda,Anual Peso Bruto KG,Anual ID Cliente,Anual Pedido,Grupo MCCAIN,Canal
0,2020,3875756.83,266181.89,625,6722,0.00,0.00,0.00,0.00,73.69,60.12,21.44,31.67,MCCAIN VAREJO,AS
1,2021,5082759.67,366293.45,697,8197,31.14,37.61,11.52,21.94,32.45,16.35,8.90,7.98,MCCAIN VAREJO,AS
2,2022,6731966.43,426198.14,759,8851,32.45,16.35,8.90,7.98,0.00,0.00,0.00,0.00,MCCAIN VAREJO,AS


Canal: B2C x MCCAIN FOOD


...

,Ano,Total Venda,Peso Bruto KG,ID Cliente,Pedido,Perc Total Venda,Perc Peso Bruto KG,Perc ID Cliente,Perc Pedido,Anual Total Venda,Anual Peso Bruto KG,Anual ID Cliente,Anual Pedido,Grupo MCCAIN,Canal
0,2020,4982.04,283.44,99,242,0.00,0.00,0.00,0.00,148.76,785.73,-20.20,-26.86,MCCAIN VAREJO,B2C
1,2021,32996.22,8556.28,195,435,562.30,2918.73,96.97,79.75,-62.44,-70.66,-59.49,-59.31,MCCAIN VAREJO,B2C
2,2022,12393.16,2510.50,79,177,-62.44,-70.66,-59.49,-59.31,0.00,0.00,0.00,0.00,MCCAIN VAREJO,B2C


Canal: FS x MCCAIN FOOD


,Ano,Total Venda,Peso Bruto KG,ID Cliente,Pedido,Perc Total Venda,Perc Peso Bruto KG,Perc ID Cliente,Perc Pedido,Anual Total Venda,Anual Peso Bruto KG,Anual ID Cliente,Anual Pedido,Grupo MCCAIN,Canal
0,2018,8968502.61,1160497.57,1173,27167,0.00,0.00,0.00,0.00,-27.21,-60.70,-20.97,-54.58,MCCAIN FOOD,FS
1,2019,10768884.88,1205769.63,1227,32141,20.07,3.90,4.60,18.31,-39.38,-62.17,-24.45,-61.61,MCCAIN FOOD,FS
2,2020,5295099.12,492541.85,972,14379,-50.83,-59.15,-20.78,-55.26,23.28,-7.40,-4.63,-14.19,MCCAIN FOOD,FS
3,2021,4884641.03,385248.54,795,11912,-7.75,-21.78,-18.21,-17.16,33.64,18.39,16.60,3.58,MCCAIN FOOD,FS
4,2022,6527731.73,456109.39,927,12339,33.64,18.39,16.60,3.58,0.00,0.00,0.00,0.00,MCCAIN FOOD,FS


Canal: FS x MCCAIN VAREJO


,Ano,Total Venda,Peso Bruto KG,ID Cliente,Pedido,Perc Total Venda,Perc Peso Bruto KG,Perc ID Cliente,Perc Pedido,Anual Total Venda,Anual Peso Bruto KG,Anual ID Cliente,Anual Pedido,Grupo MCCAIN,Canal
0,2020,138027.20,14469.80,199,599,0.00,0.00,0.00,0.00,158.00,38.42,40.70,60.77,MCCAIN VAREJO,FS
1,2021,573512.75,61598.87,432,2193,315.51,325.71,117.09,266.11,-37.91,-67.49,-35.19,-56.09,MCCAIN VAREJO,FS
2,2022,356106.46,20028.47,280,963,-37.91,-67.49,-35.19,-56.09,0.00,0.00,0.00,0.00,MCCAIN VAREJO,FS


In [19]:
def Classificacao(val):
    
    if val<=80:
        
        tipo='A'
        
        pass
    
    elif val<=95:
        
        tipo='B'
        
        pass
    
    
    else:
        
        tipo='C'
        
        pass
    
    return tipo
    
    pass

In [20]:
tabela='Curva ABC'

col='Total Venda'

dados_df=pd.DataFrame()

for f in df['Grupos']['Grupo MCCAIN'].tolist():
    
    df[tabela]=df['Estatico'].loc[(df['Estatico']['Grupo MCCAIN']==f)&(df['Estatico']['Ano']==ano_atual)]
            
    df[tabela]=df[tabela].groupby(['Grupo MCCAIN','Produto'],as_index=False).agg({col:'sum'})
    
    df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)
    
    df[tabela]['Total']=df[tabela][col].sum()
    
    df[tabela]['Perc']=round(df[tabela][col]/df[tabela]['Total'],4)*100
    
    valores=[]
    
    total=0
    
    for i in range(0,len(df[tabela])):
        
        total+=df[tabela].loc[i,col].sum()
        
        valores.append(total)
        
        pass
    
    df[tabela]['Acumulado']=valores
    
    df[tabela]['Perc Acumulado']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100
    
    df[tabela]['Classificação']=df[tabela]['Perc Acumulado'].apply(Classificacao)
    
    dados_df=pd.concat([dados_df,df[tabela]],axis=0,ignore_index=True)
    
    display(df[tabela])
    
    pass

df[tabela]=dados_df


,Grupo MCCAIN,Produto,Total Venda,Total,Perc,Acumulado,Perc Acumulado,Classificação
0,MCCAIN FOOD,"CONGELADO BATATA MCCAIN 9MM TRADICIONAL 2,5",2134935.53,7255113.55,29.43,2134935.53,29.43,A
1,MCCAIN FOOD,"CONGELADO BATATA MCCAIN BINT 9MM (2,5KG)",834290.84,7255113.55,11.50,2969226.37,40.93,A
2,MCCAIN FOOD,"CONGELADO BATATA MCCAIN ONE FRY (2,5KG)",641125.77,7255113.55,8.84,3610352.14,49.76,A
3,MCCAIN FOOD,CONGELADO BATATA MCCAIN SURECRISP 9MM NAC,570307.89,7255113.55,7.86,4180660.03,57.62,A
4,MCCAIN FOOD,"CONG. BATATA MCCAIN 7MM IMP (2,250 KG)",539385.69,7255113.55,7.43,4720045.72,65.06,A
5,MCCAIN FOOD,"CONGELADO BATATA MCCAIN SURECRISP 7MM 8X2,25KG",311704.00,7255113.55,4.30,5031749.72,69.35,A
6,MCCAIN FOOD,"CONGELADO BATATA MCCAIN CANOA FRYNDIP (2,5KG)",264446.61,7255113.55,3.64,5296196.33,73.00,A
7,MCCAIN FOOD,"CONG. BATATA MCCAIN SMILES NAC (1,5KG)",259439.24,7255113.55,3.58,5555635.57,76.58,A
8,MCCAIN FOOD,"CONGELADO BATATA MCCAIN RUSTICA (2,5KG)",256647.10,7255113.55,3.54,5812282.67,80.11,B
9,MCCAIN FOOD,"CONGELADO BATATA MCCAIN FARMPAC 9MM 2,5KG",221181.56,7255113.55,3.05,6033464.23,83.16,B


,Grupo MCCAIN,Produto,Total Venda,Total,Perc,Acumulado,Perc Acumulado,Classificação
0,MCCAIN VAREJO,"CONG. BATATA MCCAIN 9MM TRAD 1,5KG (10X1,5KG)",1886279.36,7100466.05,26.57,1886279.36,26.57,A
1,MCCAIN VAREJO,CONG. BATATA MCCAIN 9MM TRAD 720G (12X720GR),1104856.79,7100466.05,15.56,2991136.15,42.13,A
2,MCCAIN VAREJO,CONG. BATATA MCCAIN 9MM TRAD 720G (16X720GR),774295.25,7100466.05,10.90,3765431.40,53.03,A
3,MCCAIN VAREJO,CONG. BATATA MCCAIN SMILES 400G (22X400GR),723178.84,7100466.05,10.18,4488610.24,63.22,A
4,MCCAIN VAREJO,CONG. BATATA MCCAIN FORNO 600G (16X600GR),445978.21,7100466.05,6.28,4934588.45,69.50,A
5,MCCAIN VAREJO,CONG. BATATA MCCAIN 9MM TRAD 720G (12X720GR) I...,421442.98,7100466.05,5.94,5356031.43,75.43,A
6,MCCAIN VAREJO,CONG. BATATA MCCAIN BINT 9MM (8X2KG),364717.39,7100466.05,5.14,5720748.82,80.57,B
7,MCCAIN VAREJO,CONG. ANEL DE CEBOLA BRAZIL 400G (18X400GR),325176.13,7100466.05,4.58,6045924.95,85.15,B
8,MCCAIN VAREJO,CONG. BATATA MCCAIN CANOA 500G (18X500GR),216062.60,7100466.05,3.04,6261987.55,88.19,B
9,MCCAIN VAREJO,CONG. BATATA MCCAIN 9MM TRAD 400G (20X400GR),210755.00,7100466.05,2.97,6472742.55,91.16,B


In [21]:
tabela='Curva ABC Geral'

col='Total Venda'

colunas=['Produto']

dados_df=pd.DataFrame()

for i,c in enumerate(colunas):

    df[tabela]=df['Estatico'].loc[(df['Estatico']['Ano']==ano_atual)]
    
    if i==0:

        df[tabela]=df[tabela].groupby([c],as_index=False).agg({col:'sum'})
        
        pass
    
    else:
        
        indice=dados_df.loc[dados_df['Classificação']!='C',colunas[i-1]].unique().tolist()
                        
        df[tabela]=df[tabela].loc[df[tabela][colunas[i-1]].isin(indice)].groupby(colunas[:i+1],as_index=False).agg({col:'sum'})
        
        pass

    df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

    df[tabela]['Total']=df[tabela][col].sum()

    df[tabela]['Perc']=round(df[tabela][col]/df[tabela]['Total'],4)*100

    valores=[]

    total=0

    for i in range(0,len(df[tabela])):

        total+=df[tabela].loc[i,col].sum()

        valores.append(total)

        pass

    df[tabela]['Acumulado']=valores

    df[tabela]['Perc Acumulado']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

    df[tabela]['Classificação']=df[tabela]['Perc Acumulado'].apply(Classificacao)

    dados_df=pd.concat([dados_df,df[tabela]],axis=0,ignore_index=True)
        
    pass

df[tabela]

#df[tabela].to_excel(f'{tabela}.xlsx',index=False)

,Produto,Total Venda,Total,Perc,Acumulado,Perc Acumulado,Classificação
0,"CONGELADO BATATA MCCAIN 9MM TRADICIONAL 2,5",2134935.53,14355579.60,14.87,2134935.53,14.87,A
1,"CONG. BATATA MCCAIN 9MM TRAD 1,5KG (10X1,5KG)",1886279.36,14355579.60,13.14,4021214.89,28.01,A
2,CONG. BATATA MCCAIN 9MM TRAD 720G (12X720GR),1104856.79,14355579.60,7.70,5126071.68,35.71,A
3,"CONGELADO BATATA MCCAIN BINT 9MM (2,5KG)",834290.84,14355579.60,5.81,5960362.52,41.52,A
4,CONG. BATATA MCCAIN 9MM TRAD 720G (16X720GR),774295.25,14355579.60,5.39,6734657.77,46.91,A
...,...,...,...,...,...,...,...
58,CONG. BATATA MCCAIN RUSTICA 720G (12X720GR) IM...,2174.46,14355579.60,0.02,14353174.29,99.98,C
59,"CONGELADO BATATA MCCAIN 7MM (2,250 KG)",1448.12,14355579.60,0.01,14354622.41,99.99,C
60,CONG. PICKERS ARANCINI MCCAIN CRU NAPOLITANO 1...,621.42,14355579.60,0.00,14355243.83,100.00,C
61,CONG. BATATA MCCAIN SMILES 500G (18X500GR),206.69,14355579.60,0.00,14355450.52,100.00,C


In [22]:
# top 5

tabela='TOP 5'

col='Total Venda'

colunas=['Linha','Produto']

dados_df=pd.DataFrame()

for i,c in enumerate(colunas):

    df[tabela]=df['Estatico'].loc[(df['Estatico']['Ano']==ano_atual)]
    
    if i==0:

        df[tabela]=df[tabela].groupby([c],as_index=False).agg({col:'sum'})
        
        pass
    
    else:
        
        indice=dados_df[colunas[i-1]].head().tolist()
        
        dados_df=pd.DataFrame()
                                
        df[tabela]=df[tabela].loc[df[tabela][colunas[i-1]].isin(indice)].groupby(colunas[:i+1],as_index=False).agg({col:'sum'})
        
        pass

    df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

    df[tabela]['Total']=df[tabela][col].sum()

    df[tabela]['Perc']=round(df[tabela][col]/df[tabela]['Total'],4)*100

    valores=[]

    total=0

    for i in range(0,len(df[tabela])):

        total+=df[tabela].loc[i,col].sum()

        valores.append(total)

        pass

    df[tabela]['Acumulado']=valores

    df[tabela]['Perc Acumulado']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

    df[tabela]['Classificação']=df[tabela]['Perc Acumulado'].apply(Classificacao)
        
    dados_df=pd.concat([dados_df,df[tabela]],axis=0,ignore_index=True)
        
    pass

dados_df

,Linha,Produto,Total Venda,Total,Perc,Acumulado,Perc Acumulado,Classificação
0,BATATA GRADO A,"CONGELADO BATATA MCCAIN 9MM TRADICIONAL 2,5",2134935.53,12263541.85,17.41,2134935.53,17.41,A
1,BATATA PALITO,"CONG. BATATA MCCAIN 9MM TRAD 1,5KG (10X1,5KG)",1886279.36,12263541.85,15.38,4021214.89,32.79,A
2,BATATA PALITO,CONG. BATATA MCCAIN 9MM TRAD 720G (12X720GR),1104856.79,12263541.85,9.01,5126071.68,41.80,A
3,BATATA GRADO B,"CONGELADO BATATA MCCAIN BINT 9MM (2,5KG)",834290.84,12263541.85,6.80,5960362.52,48.60,A
4,BATATA PALITO,CONG. BATATA MCCAIN 9MM TRAD 720G (16X720GR),774295.25,12263541.85,6.31,6734657.77,54.92,A
5,BATATA ESPECIALIDADE,CONG. BATATA MCCAIN SMILES 400G (22X400GR),723178.84,12263541.85,5.90,7457836.61,60.81,A
6,BATATA GRADO B,"CONGELADO BATATA MCCAIN ONE FRY (2,5KG)",641125.77,12263541.85,5.23,8098962.38,66.04,A
7,PALITO EXTRA CROCANT,CONGELADO BATATA MCCAIN SURECRISP 9MM NAC,570307.89,12263541.85,4.65,8669270.27,70.69,A
8,BATATA PALITO,CONG. BATATA MCCAIN FORNO 600G (16X600GR),445978.21,12263541.85,3.64,9115248.48,74.33,A
9,BATATA PALITO,CONG. BATATA MCCAIN 9MM TRAD 720G (12X720GR) I...,421442.98,12263541.85,3.44,9536691.46,77.76,A


In [23]:
df[tabela].groupby(['Classificação'],as_index=False).agg({'Produto':'count','Total Venda':'sum','Perc':'sum'})

,Classificação,Produto,Total Venda,Perc
0,A,10,9536691.46,77.77
1,B,8,1979396.53,16.14
2,C,18,747453.86,6.10


In [24]:
writer=pd.ExcelWriter('MCCAIN.xlsx',engine='xlsxwriter')

for l in df.keys():
    
    if l in querys.keys():
        
        continue
    
    df[l].to_excel(writer,sheet_name=l,index=False,encoding='UTF-8')
    
    pass

writer.close()
